# memo

# import など準備

## import, MySQL 接続

In [2]:
# import
import sys
import os
import re
import datetime as dt
import time
import importlib
import logging
import numpy as np
import pandas as pd
from robobrowser import RoboBrowser
# from robobrowser.browser import RoboState
from retry import retry
from dateutil.parser import parse
from datetime import datetime

import stock

In [ ]:
importlib.reload(stock)

In [3]:
# pandas の最大表示列数を設定 (max_rows で表示行数の設定も可能)
pd.set_option('display.max_columns', 30)

In [4]:
sql = stock.sql()

## 比較参照用、株プロ決算の読み込み

In [3]:
table_name = 'kabupro_kessan'

In [6]:
kabupro = sql.read_table(table_name)

In [10]:
kabupro

,証券コード,企業名,会計基準,連結個別,決算期,決算期間,期首,期末,名寄前勘定科目 (売上高欄),売上高,営業利益,経常利益,純利益,一株当り純利益,希薄化後一株当り純利益,純資産又は株主資本,総資産,一株当り純資産,営業キャッシュフロー,投資キャッシュフロー,財務キャッシュフロー,情報公開日 (更新日)
0,1301,極洋,日本基準,個別,2013年3月期,第2四半期,2012-04-01,2012-09-30,売上高,7.654600e+10,1.530000e+08,3.530000e+08,7.900000e+07,0.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-10-21
1,1301,極洋,日本基準,個別,2013年3月期,通期,2012-04-01,2013-03-31,売上高,1.606100e+11,9.900000e+08,1.392000e+09,8.060000e+08,7.68,NaN,1.481800e+10,7.049200e+10,141.08,NaN,NaN,NaN,2014-05-09
2,1301,極洋,日本基準,個別,2014年3月期,第2四半期,2013-04-01,2013-09-30,売上高,8.816100e+10,1.367000e+09,1.457000e+09,1.834000e+09,17.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-27
3,1301,極洋,日本基準,個別,2014年3月期,通期,2013-04-01,2014-03-31,売上高,1.920260e+11,2.471000e+09,2.546000e+09,1.806000e+09,17.20,16.77,1.616400e+10,7.038100e+10,153.90,NaN,NaN,NaN,2015-05-08
4,1301,極洋,日本基準,個別,2015年3月期,第2四半期,2014-04-01,2014-09-30,売上高,9.947600e+10,8.900000e+08,9.500000e+08,2.504000e+09,23.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-26
5,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,1.894000e+09,2.042000e+09,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2015-05-08
6,1301,極洋,日本基準,個別,2015年3月期,通期,2014-04-01,2015-03-31,売上高,2.120860e+11,1.894000e+09,2.042000e+09,2.139000e+09,20.37,18.78,1.769500e+10,7.351900e+10,168.48,NaN,NaN,NaN,2016-05-09
7,1301,極洋,日本基準,個別,2016年3月期,通期,2015-04-01,2016-03-31,売上高,2.163500e+11,2.211000e+09,2.432000e+09,1.259000e+09,119.87,110.51,1.781700e+10,7.814000e+10,1696.42,NaN,NaN,NaN,2017-05-11
8,1301,極洋,日本基準,個別,2017年3月期,通期,2016-04-01,2017-03-31,売上高,2.280830e+11,3.187000e+09,2.835000e+09,2.194000e+09,208.94,192.95,2.011700e+10,8.130600e+10,1915.42,NaN,NaN,NaN,2017-05-11
9,1301,極洋,日本基準,連結,2013年3月期,第1四半期,2012-04-01,2012-06-30,売上高,4.319100e+10,4.690000e+08,6.770000e+08,3.350000e+08,3.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-05


In [5]:
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

In [43]:
kabupro.ix[14]
# 株プロにしか無い項目: 希薄化後一株当り純利益, 純資産又は株主資本, 営業キャッシュフロー, 投資キャッシュフロー, 財務キャッシュフロー

証券コード                            1301
企業名                                極洋
会計基準                             日本基準
連結個別                               連結
決算期                          2014年3月期
決算期間                            第2四半期
期首                2013-04-01 00:00:00
期末                2013-09-30 00:00:00
名寄前勘定科目 (売上高欄)                    売上高
売上高                        9.4101e+10
営業利益                        1.404e+09
経常利益                          1.5e+09
純利益                          1.88e+09
一株当り純利益                          17.9
希薄化後一株当り純利益                       NaN
純資産又は株主資本                         NaN
総資産                               NaN
一株当り純資産                           NaN
営業キャッシュフロー                        NaN
投資キャッシュフロー                        NaN
財務キャッシュフロー                        NaN
情報公開日 (更新日)       2014-11-07 00:00:00
Name: 14, dtype: object

# 銘柄コードリスト

In [16]:
domestic_stock_table = sql.read_table('domestic_stock_table')

In [17]:
domestic_stock_table

,date,code,name,market,code_33,category_33,code_17,category_17,code_scale,scale
0,20171031,1301,極洋,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
1,20171031,1332,日本水産,市場第一部（内国株）,50,水産・農林業,1,食品,4,TOPIX Mid400
2,20171031,1333,マルハニチロ,市場第一部（内国株）,50,水産・農林業,1,食品,4,TOPIX Mid400
3,20171031,1352,ホウスイ,市場第一部（内国株）,6050,卸売業,13,商社・卸売,7,TOPIX Small 2
4,20171031,1376,カネコ種苗,市場第一部（内国株）,50,水産・農林業,1,食品,7,TOPIX Small 2
5,20171031,1377,サカタのタネ,市場第一部（内国株）,50,水産・農林業,1,食品,6,TOPIX Small 1
6,20171031,1379,ホクト,市場第一部（内国株）,50,水産・農林業,1,食品,6,TOPIX Small 1
7,20171031,1380,秋川牧園,JASDAQ(スタンダード・内国株）,50,水産・農林業,1,食品,-,-
8,20171031,1381,アクシーズ,JASDAQ(スタンダード・内国株）,50,水産・農林業,1,食品,-,-
9,20171031,1382,ホーブ,JASDAQ(スタンダード・内国株）,50,水産・農林業,1,食品,-,-


In [18]:
code_list = list(domestic_stock_table['code'])

In [74]:
code_list[-10:]

[9987, 9989, 9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997]

In [20]:
len(code_list)

3548

In [75]:
start_index = 30
increase_number = 10
# end_index = start_index + increase_number
end_index = len(code_list)

reading_code = code_list[start_index : end_index]
print(reading_code[-10:])
print('Next start from {0}'.format(start_index + increase_number))

[9987, 9989, 9990, 9991, 9992, 9993, 9994, 9995, 9996, 9997]
Next start from 40


# 保存した html ファイルからテーブル属性のみ読み込み、整形

__TODO__

In [5]:
code = 7203

In [22]:
# 保存した html からテーブル属性を読み込み
tables = pd.read_html('/Users/Really/Stockyard/_kabutan_html/kabutan_{0}.html'.format(code), header=0)

## リスト、テーブルの概要

In [7]:
len(tables)

29

In [8]:
len(tables[12])

52

In [9]:
len(tables[12].columns)

14

In [23]:
# tables[11] 通期業績

# 全ての列項目がnullの行を除去
tables[11] = tables[11][~tables[11].isnull().all(axis=1)].reset_index(drop=True)

# 予想値と前期比の行を除去
tables[11] = tables[11][~((tables[11]['決算期'].str.contains('予')) | (tables[11]['決算期'].str.contains('前期比')))].reset_index(drop=True)

# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[11][['会計基準', '決算期']] = pd.DataFrame(list(tables[11]['決算期'].str.split(' ')))

# 列の並び替え
tables[11] = tables[11][['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日']]

# 100万円単位換算
tables[11][['売上高', '営業益', '経常益', '最終益']] = tables[11][['売上高', '営業益', '経常益', '最終益']].apply(lambda x: x * 1000000)

# 型変換
tables[11]['１株配'] = tables[11]['１株配'].astype(float)

# 日付のパース、datetime.dateへの型変換
# tables[11]['決算期'] = tables[11]['決算期'].apply(lambda x: datetime.strptime(x, '%Y.%m').date()) # 日付ではないので文字列のままの方がいいかも？
tables[11]['発表日'] = tables[11]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[11]['発表日'] = pd.to_datetime(tables[11]['発表日'], format='%Y-%m-%d')
# tables[11]['決算期'] = pd.to_datetime(tables[11]['決算期'], format='%Y-%m-%d')

# tables[12] 業績予想

tables[12].columns = ['会計基準', '決算期', '発表日', 
                                   '結合修正方向', '売上高修正方向', '営業益修正方向', '経常益修正方向', '最終益修正方向', '修正配当修正方向', 
                                   '予想売上高', '予想営業益', '予想経常益', '予想最終益', '予想修正配当',]

# 不要行、不要列の削除、並び替え
# 実績(と修正配当)はいる?いらない?
# 実績の発表と同時に次の予想が出ているのでやっぱりここではいらないのかな?
tables[12] = tables[12].ix[tables[12].index % 2 == 0, ['会計基準', '決算期', '予想売上高', '予想営業益', '予想経常益', '予想最終益', '発表日']].reset_index(drop=True)
tables[12] = tables[12].ix[tables[12]['決算期'] != '実績']

# 決算期の NaN 埋め
tables[12]['決算期'] = tables[12]['決算期'].fillna(method='ffill')

# 100万円単位換算
tables[12][['予想売上高', '予想営業益', '予想経常益', '予想最終益']] = tables[12][['予想売上高', '予想営業益', '予想経常益', '予想最終益']].apply(lambda x: x * 1000000)

# 日付のパース、datetime.dateへの型変換
# tables[12]['決算期'] = tables[12]['決算期'].apply(lambda x: datetime.strptime(x, '%Y.%m').date()) # 日付ではないので文字列のままの方がいいかも？
tables[12]['発表日'] = tables[12]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[12]['発表日'] = pd.to_datetime(tables[12]['発表日'], format='%Y-%m-%d')
# tables[12]['決算期'] = pd.to_datetime(tables[12]['決算期'], format='%Y-%m-%d')

# 修正配当用の処理なので不要
# '－'  を NaN に置換
# tables[12].loc[~tables[12]['修正配当'].str.isnumeric(), '修正配当'] = np.nan
# 型変換
# tables[12]['修正配当'] = tables[12]['修正配当'].astype(float)

# tables[26] ３ヵ月業績の推移【実績】(過去5年 + 前年同期比) 累積ではなく差分

'''
不要かな？
ちょっと株プロと見比べてみよう → １株益の値が揃わない
修正発表があった項目は上書きされてしまっていると思われる
修正の可能性を考えなければ累積の株プロよりこちらの方が使いやすいかも
株プロで差分を作成するべきか?
前年同期比はいらなそう
'''

# 全ての列項目がnullの行を除去
tables[26] = tables[26][~tables[26].isnull().all(axis=1)].reset_index(drop=True)

# 前年同期比の行を除去
tables[26] = tables[26][~tables[26]['決算期'].str.contains('前年同期比')].reset_index(drop=True)

# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[26][['会計基準', '四半期期首']] = pd.DataFrame(list(tables[26]['決算期'].str.split(' ')))

# 列の並び替え
tables[26] = tables[26][['会計基準', '四半期期首', '売上高', '営業益', '経常益', '最終益', '１株益', '売上営業損益率', '発表日']]

tables[26].columns = ['会計基準', '四半期期首', '四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益', '四半期１株益', '四半期売上営業損益率', '発表日']

# 100万円単位換算
tables[26][['四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益']] = tables[26][['四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益']].apply(lambda x: x * 1000000)

# 日付のパース、datetime.dateへの型変換
tables[26]['四半期期首'] = tables[26]['四半期期首'].apply(lambda x: parse(x.replace('-', '.'), yearfirst=True).date())
tables[26]['発表日'] = tables[26]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
tables[26]['四半期期首'] = pd.to_datetime(tables[26]['四半期期首'], format='%Y-%m-%d')
tables[26]['発表日'] = pd.to_datetime(tables[26]['発表日'], format='%Y-%m-%d')

# tables[27] 財務 【実績】

'''
期間は株プロよりこちらの方が長い
修正発表があった項目は上書きされてしまっていると思われる
'''

# 7203 トヨタ 元の値が '90/08/01' で明らかにおかしい。これどうしよう。。。
# tables[27].ix[2, '発表日'] = '99/08/01'

# 全ての列項目がnullの行を除去
tables[27] = tables[27][~tables[27].isnull().all(axis=1)].reset_index(drop=True)

# 決算期列の要素を会計基準と決算期に分割、それぞれの列に代入(同時に会計基準列を新規作成)
tables[27][['会計基準', '決算期']] = pd.DataFrame(list(tables[27]['決算期'].str.split(' ')))

# 列の並び替え
tables[27] = tables[27][['会計基準', '決算期', '１株純資産', '自己資本比率', '総資産', '自己資本', '剰余金', '有利子負債倍率', '発表日']]

# 決算期が 'yyyy.mm' 表記ではない行は確定決算前と判断して削除
tables[27] = tables[27][tables[27]['決算期'].str.contains('\d\d\d\d.\d\d')]

# '－'  を NaN に置換
# .str を2回も使わないといけないのはなんだか。。。
tables[27].loc[~tables[27]['１株純資産'].str.replace('.', '').str.isnumeric(), '１株純資産'] = np.nan
tables[27].loc[~tables[27]['有利子負債倍率'].str.replace('.', '').str.isnumeric(), '有利子負債倍率'] = np.nan

# 型変換
tables[27][['１株純資産', '有利子負債倍率']] = tables[27][['１株純資産', '有利子負債倍率']].astype(float)

# 100万円単位換算
tables[27][['総資産', '自己資本', '剰余金']] = tables[27][['総資産', '自己資本', '剰余金']].apply(lambda x: x * 1000000)

# 日付のパース、datetime.dateへの型変換
# tables[27]['決算期'] = tables[27]['決算期'].apply(lambda x: parse(x.replace('-', '.'), yearfirst=True).date()) # 日付ではないので文字列のままの方がいいかも？
tables[27]['発表日'] = tables[27]['発表日'].apply(lambda x: parse(x, yearfirst=True).date())
# pandasのTimestampへの型変換
# tables[27]['決算期'] = pd.to_datetime(tables[27]['決算期'], format='%Y-%m-%d')
tables[27]['発表日'] = pd.to_datetime(tables[27]['発表日'], format='%Y-%m-%d')

# 内容の確認

In [8]:
# tables[11] 通期業績
tables[11]
# 株プロに無い項目: １株配

,会計基準,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,連,1999.03,1.274901e+13,7.749450e+11,7.718850e+11,3.561800e+11,94.2,23.0,1999-05-24
1,連,2000.03,1.287956e+13,7.759820e+11,7.970580e+11,4.067980e+11,110.0,24.0,2000-05-17
2,連,2001.03,1.342442e+13,8.701310e+11,9.722730e+11,4.712950e+11,127.9,25.0,2001-05-16
3,連,2002.03,1.510630e+13,1.123470e+12,1.113524e+12,6.158240e+11,170.7,28.0,2002-05-13
4,U,2003.03,1.550155e+13,1.271646e+12,1.226652e+12,7.509420e+11,211.3,36.0,2003-05-08
5,U,2004.03,1.729476e+13,1.666890e+12,1.765793e+12,1.162098e+12,342.9,45.0,2004-05-11
6,U,2005.03,1.855153e+13,1.672187e+12,1.754637e+12,1.171260e+12,355.4,65.0,2005-05-10
7,U,2006.03,2.103691e+13,1.878342e+12,2.087360e+12,1.372180e+12,421.8,90.0,2006-05-10
8,U,2007.03,2.394809e+13,2.238683e+12,2.382516e+12,1.644032e+12,512.1,120.0,2007-05-09
9,U,2008.03,2.628924e+13,2.270375e+12,2.437222e+12,1.717879e+12,540.6,140.0,2008-05-08


In [9]:
# tables[12] 業績予想
tables[12]

,会計基準,決算期,予想売上高,予想営業益,予想経常益,予想最終益,発表日
0,U,2013.03,2.200000e+13,1.000000e+12,1.160000e+12,7.600000e+11,2012-05-09
1,U,2013.03,2.130000e+13,1.050000e+12,1.180000e+12,7.800000e+11,2012-11-05
2,U,2013.03,2.180000e+13,1.150000e+12,1.290000e+12,8.600000e+11,2013-02-05
4,U,2014.03,2.350000e+13,1.800000e+12,1.890000e+12,1.370000e+12,2013-05-08
5,U,2014.03,2.400000e+13,1.940000e+12,2.030000e+12,1.480000e+12,2013-08-02
6,U,2014.03,2.500000e+13,2.200000e+12,2.290000e+12,1.670000e+12,2013-11-06
7,U,2014.03,2.550000e+13,2.400000e+12,2.530000e+12,1.900000e+12,2014-02-04
9,U,2015.03,2.570000e+13,2.300000e+12,2.390000e+12,1.780000e+12,2014-05-08
10,U,2015.03,2.650000e+13,2.500000e+12,2.700000e+12,2.000000e+12,2014-11-05
11,U,2015.03,2.700000e+13,2.700000e+12,2.920000e+12,2.130000e+12,2015-02-04


In [10]:
# tables[26] ３ヵ月業績の推移
tables[26]
# 株プロに無い項目: 売上営業損益率 = 営業益 / 売上高?

,会計基準,四半期期首,四半期売上高,四半期営業益,四半期経常益,四半期最終益,四半期１株益,四半期売上営業損益率,発表日
0,U,2012-10-12,5.318752e+12,1.247570e+11,1.312490e+11,9.991400e+10,31.6,2.3,2013-02-05
1,U,2013-01-03,5.837086e+12,5.023810e+11,4.778630e+11,3.139800e+11,99.1,8.6,2013-05-08
2,U,2013-04-06,6.255319e+12,6.633830e+11,7.241630e+11,5.621940e+11,177.4,10.6,2013-08-02
3,U,2013-07-09,6.282166e+12,5.920920e+11,6.193620e+11,4.384290e+11,138.4,9.4,2013-11-06
4,U,2013-10-12,6.585044e+12,6.005090e+11,6.789560e+11,5.254640e+11,165.8,9.1,2014-02-04
5,U,2014-01-03,6.569382e+12,4.361280e+11,4.185990e+11,2.970320e+11,93.7,6.6,2014-05-08
6,U,2014-04-06,6.390688e+12,6.927280e+11,7.718260e+11,5.877740e+11,185.4,10.8,2014-08-05
7,U,2014-07-09,6.554909e+12,6.592180e+11,7.373480e+11,5.390620e+11,170.3,10.1,2014-11-05
8,U,2014-10-12,7.170017e+12,7.628800e+11,8.464610e+11,6.000270e+11,189.7,10.6,2015-02-04
9,U,2015-01-03,7.118907e+12,6.357380e+11,5.371930e+11,4.464750e+11,141.3,8.9,2015-05-08


In [24]:
# tables[27] 財務
tables[27]
# 株プロに無い項目: 自己資本比率, 自己資本, 剰余金, 有利子負債倍率

,会計基準,決算期,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率,発表日
0,連,1998.03,1583.20,43.5,1.385436e+13,6.021895e+12,5.213340e+12,NaN,1998-08-01
1,連,1999.03,1642.90,41.9,1.475331e+13,6.175937e+12,5.367879e+12,NaN,1999-08-01
2,連,2000.03,1841.50,41.3,1.646905e+13,6.796666e+12,6.014125e+12,0.64,2000-05-17
3,連,2001.03,1956.57,40.6,1.751943e+13,7.114567e+12,6.162656e+12,0.63,2001-05-16
4,連,2002.03,2059.94,36.8,1.988894e+13,7.325072e+12,6.527956e+12,0.79,2002-05-13
5,U,2003.03,2063.43,35.3,2.015297e+13,7.121000e+12,7.219896e+12,1.02,2003-05-08
6,U,2004.03,2456.08,37.1,2.204023e+13,8.178567e+12,8.326215e+12,0.92,2004-05-11
7,U,2005.03,2767.67,37.2,2.433501e+13,9.044950e+12,9.332176e+12,0.95,2005-05-10
8,U,2006.03,3257.63,36.8,2.873160e+13,1.056045e+13,1.045979e+13,0.98,2006-05-10
9,U,2007.03,3701.17,36.3,3.257478e+13,1.183609e+13,1.176471e+13,1.02,2007-05-09


In [23]:
tables[11].columns

Index(['決算期', '修正日', '修正方向', '売上高', '営業益', '経常益', '最終益', '修正配当', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13'],
      dtype='object')

In [23]:
tables[12].columns

Index(['決算期', '修正日', '修正方向', '売上高', '営業益', '経常益', '最終益', '修正配当', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13'],
      dtype='object')

In [373]:
tables[26].columns

Index(['会計基準', '四半期期首', '売上高', '営業益', '経常益', '最終益', '１株益', '売上営業損益率', '発表日'], dtype='object')

In [208]:
tables[27].columns

Index(['決算期', '１株純資産', '自己資本比率', '総資産', '自己資本', '剰余金', '有利子負債倍率', '発表日',
       '会計基準'],
      dtype='object')

In [12]:
tables[11].dtypes

会計基準            object
決算期             object
売上高            float64
営業益            float64
経常益            float64
最終益            float64
１株益            float64
１株配            float64
発表日     datetime64[ns]
dtype: object

In [13]:
tables[12].dtypes

会計基準             object
決算期              object
予想売上高           float64
予想営業益           float64
予想経常益           float64
予想最終益           float64
発表日      datetime64[ns]
dtype: object

In [14]:
tables[26].dtypes

会計基準                  object
四半期期首         datetime64[ns]
四半期売上高               float64
四半期営業益               float64
四半期経常益               float64
四半期最終益               float64
四半期１株益               float64
四半期売上営業損益率           float64
発表日           datetime64[ns]
dtype: object

In [15]:
tables[27].dtypes

会計基準               object
決算期                object
１株純資産             float64
自己資本比率            float64
総資産               float64
自己資本              float64
剰余金               float64
有利子負債倍率           float64
発表日        datetime64[ns]
dtype: object

In [25]:
len(tables[11])

19

In [26]:
len(tables[12])

21

In [27]:
len(tables[26])

20

In [28]:
len(tables[27])

20

# 結合

In [44]:
# 通期業績 & 財務
merged_1 = pd.merge(tables[11], tables[27], on=['会計基準', '決算期', '発表日'], how='outer').sort_values('発表日')

In [45]:
merged_1.columns

Index(['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日', '１株純資産',
       '自己資本比率', '総資産', '自己資本', '剰余金', '有利子負債倍率'],
      dtype='object')

In [47]:
merged_1[['会計基準', '発表日', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '１株純資産',
       '自己資本比率', '総資産', '自己資本', '剰余金', '有利子負債倍率']]

,会計基準,発表日,決算期,売上高,営業益,経常益,最終益,１株益,１株配,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率
19,連,1998-08-01,1998.03,NaN,NaN,NaN,NaN,NaN,NaN,1583.20,43.5,1.385436e+13,6.021895e+12,5.213340e+12,NaN
0,連,1999-05-24,1999.03,1.274901e+13,7.749450e+11,7.718850e+11,3.561800e+11,94.2,23.0,NaN,NaN,NaN,NaN,NaN,NaN
20,連,1999-08-01,1999.03,NaN,NaN,NaN,NaN,NaN,NaN,1642.90,41.9,1.475331e+13,6.175937e+12,5.367879e+12,NaN
1,連,2000-05-17,2000.03,1.287956e+13,7.759820e+11,7.970580e+11,4.067980e+11,110.0,24.0,1841.50,41.3,1.646905e+13,6.796666e+12,6.014125e+12,0.64
2,連,2001-05-16,2001.03,1.342442e+13,8.701310e+11,9.722730e+11,4.712950e+11,127.9,25.0,1956.57,40.6,1.751943e+13,7.114567e+12,6.162656e+12,0.63
3,連,2002-05-13,2002.03,1.510630e+13,1.123470e+12,1.113524e+12,6.158240e+11,170.7,28.0,2059.94,36.8,1.988894e+13,7.325072e+12,6.527956e+12,0.79
4,U,2003-05-08,2003.03,1.550155e+13,1.271646e+12,1.226652e+12,7.509420e+11,211.3,36.0,2063.43,35.3,2.015297e+13,7.121000e+12,7.219896e+12,1.02
5,U,2004-05-11,2004.03,1.729476e+13,1.666890e+12,1.765793e+12,1.162098e+12,342.9,45.0,2456.08,37.1,2.204023e+13,8.178567e+12,8.326215e+12,0.92
6,U,2005-05-10,2005.03,1.855153e+13,1.672187e+12,1.754637e+12,1.171260e+12,355.4,65.0,2767.67,37.2,2.433501e+13,9.044950e+12,9.332176e+12,0.95
7,U,2006-05-10,2006.03,2.103691e+13,1.878342e+12,2.087360e+12,1.372180e+12,421.8,90.0,3257.63,36.8,2.873160e+13,1.056045e+13,1.045979e+13,0.98


In [58]:
# 通期業績 & 業績予想
merged_2 = pd.merge(tables[11], tables[12], on=['会計基準', '決算期', '発表日'], how='outer').sort_values(['決算期', '発表日'])

In [59]:
merged_2.columns

Index(['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日', '予想売上高',
       '予想営業益', '予想経常益', '予想最終益'],
      dtype='object')

In [62]:
merged_2[['会計基準', '発表日', '決算期', '売上高', '予想売上高', '営業益', '予想営業益', '経常益', '予想経常益', '最終益', '予想最終益', '１株益', '１株配']]

,会計基準,発表日,決算期,売上高,予想売上高,営業益,予想営業益,経常益,予想経常益,最終益,予想最終益,１株益,１株配
0,連,1999-05-24,1999.03,1.274901e+13,NaN,7.749450e+11,NaN,7.718850e+11,NaN,3.561800e+11,NaN,94.2,23.0
1,連,2000-05-17,2000.03,1.287956e+13,NaN,7.759820e+11,NaN,7.970580e+11,NaN,4.067980e+11,NaN,110.0,24.0
2,連,2001-05-16,2001.03,1.342442e+13,NaN,8.701310e+11,NaN,9.722730e+11,NaN,4.712950e+11,NaN,127.9,25.0
3,連,2002-05-13,2002.03,1.510630e+13,NaN,1.123470e+12,NaN,1.113524e+12,NaN,6.158240e+11,NaN,170.7,28.0
4,U,2003-05-08,2003.03,1.550155e+13,NaN,1.271646e+12,NaN,1.226652e+12,NaN,7.509420e+11,NaN,211.3,36.0
5,U,2004-05-11,2004.03,1.729476e+13,NaN,1.666890e+12,NaN,1.765793e+12,NaN,1.162098e+12,NaN,342.9,45.0
6,U,2005-05-10,2005.03,1.855153e+13,NaN,1.672187e+12,NaN,1.754637e+12,NaN,1.171260e+12,NaN,355.4,65.0
7,U,2006-05-10,2006.03,2.103691e+13,NaN,1.878342e+12,NaN,2.087360e+12,NaN,1.372180e+12,NaN,421.8,90.0
8,U,2007-05-09,2007.03,2.394809e+13,NaN,2.238683e+12,NaN,2.382516e+12,NaN,1.644032e+12,NaN,512.1,120.0
9,U,2008-05-08,2008.03,2.628924e+13,NaN,2.270375e+12,NaN,2.437222e+12,NaN,1.717879e+12,NaN,540.6,140.0


In [53]:
# 通期業績 & 四半期業績
merged_3 = pd.merge(tables[11], tables[26], on=['会計基準', '発表日'], how='outer').sort_values(['発表日'])

In [54]:
merged_3.columns

Index(['会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日', '四半期期首',
       '四半期売上高', '四半期営業益', '四半期経常益', '四半期最終益', '四半期１株益', '四半期売上営業損益率'],
      dtype='object')

In [56]:
merged_3[['会計基準', '発表日', '決算期', '四半期期首', '売上高', '四半期売上高', '営業益', '四半期営業益', '経常益', '四半期経常益', 
          '最終益', '四半期最終益', '１株益', '四半期１株益', '１株配', '四半期売上営業損益率']]

,会計基準,発表日,決算期,四半期期首,売上高,四半期売上高,営業益,四半期営業益,経常益,四半期経常益,最終益,四半期最終益,１株益,四半期１株益,１株配,四半期売上営業損益率
0,連,1999-05-24,1999.03,NaT,1.274901e+13,NaN,7.749450e+11,NaN,7.718850e+11,NaN,3.561800e+11,NaN,94.2,NaN,23.0,NaN
1,連,2000-05-17,2000.03,NaT,1.287956e+13,NaN,7.759820e+11,NaN,7.970580e+11,NaN,4.067980e+11,NaN,110.0,NaN,24.0,NaN
2,連,2001-05-16,2001.03,NaT,1.342442e+13,NaN,8.701310e+11,NaN,9.722730e+11,NaN,4.712950e+11,NaN,127.9,NaN,25.0,NaN
3,連,2002-05-13,2002.03,NaT,1.510630e+13,NaN,1.123470e+12,NaN,1.113524e+12,NaN,6.158240e+11,NaN,170.7,NaN,28.0,NaN
4,U,2003-05-08,2003.03,NaT,1.550155e+13,NaN,1.271646e+12,NaN,1.226652e+12,NaN,7.509420e+11,NaN,211.3,NaN,36.0,NaN
5,U,2004-05-11,2004.03,NaT,1.729476e+13,NaN,1.666890e+12,NaN,1.765793e+12,NaN,1.162098e+12,NaN,342.9,NaN,45.0,NaN
6,U,2005-05-10,2005.03,NaT,1.855153e+13,NaN,1.672187e+12,NaN,1.754637e+12,NaN,1.171260e+12,NaN,355.4,NaN,65.0,NaN
7,U,2006-05-10,2006.03,NaT,2.103691e+13,NaN,1.878342e+12,NaN,2.087360e+12,NaN,1.372180e+12,NaN,421.8,NaN,90.0,NaN
8,U,2007-05-09,2007.03,NaT,2.394809e+13,NaN,2.238683e+12,NaN,2.382516e+12,NaN,1.644032e+12,NaN,512.1,NaN,120.0,NaN
9,U,2008-05-08,2008.03,NaT,2.628924e+13,NaN,2.270375e+12,NaN,2.437222e+12,NaN,1.717879e+12,NaN,540.6,NaN,140.0,NaN


In [78]:
# すべて
merged_all = pd.merge(tables[11], tables[12], on=['会計基準', '決算期', '発表日'], how='outer').sort_values(['発表日'])

In [80]:
# すべて
merged_all = pd.merge(merged_all, tables[26], on=['会計基準', '発表日'], how='outer').sort_values(['発表日'])

In [82]:
# すべて
merged_all = pd.merge(merged_all, tables[27], on=['会計基準', '決算期', '発表日'], how='outer').sort_values(['発表日'])

In [84]:
merged_all = merged_all.reset_index(drop=True)

In [90]:
merged_all.columns

Index(['発表日', '会計基準', '決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '予想売上高',
       '予想営業益', '予想経常益', '予想最終益', '四半期期首', '四半期売上高', '四半期営業益', '四半期経常益',
       '四半期最終益', '四半期１株益', '四半期売上営業損益率', '１株純資産', '自己資本比率', '総資産', '自己資本',
       '剰余金', '有利子負債倍率'],
      dtype='object')

In [91]:
merged_all = merged_all[['発表日', '決算期', '四半期期首', '会計基準', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '予想売上高',
       '予想営業益', '予想経常益', '予想最終益', '四半期売上高', '四半期営業益', '四半期経常益',
       '四半期最終益', '四半期１株益', '四半期売上営業損益率', '１株純資産', '自己資本比率', '総資産', '自己資本',
       '剰余金', '有利子負債倍率']]

In [92]:
merged_all

,発表日,決算期,四半期期首,会計基準,売上高,営業益,経常益,最終益,１株益,１株配,予想売上高,予想営業益,予想経常益,予想最終益,四半期売上高,四半期営業益,四半期経常益,四半期最終益,四半期１株益,四半期売上営業損益率,１株純資産,自己資本比率,総資産,自己資本,剰余金,有利子負債倍率
0,1998-08-01,1998.03,NaT,連,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1583.20,43.5,1.385436e+13,6.021895e+12,5.213340e+12,NaN
1,1999-05-24,1999.03,NaT,連,1.274901e+13,7.749450e+11,7.718850e+11,3.561800e+11,94.2,23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1999-08-01,1999.03,NaT,連,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1642.90,41.9,1.475331e+13,6.175937e+12,5.367879e+12,NaN
3,2000-05-17,2000.03,NaT,連,1.287956e+13,7.759820e+11,7.970580e+11,4.067980e+11,110.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1841.50,41.3,1.646905e+13,6.796666e+12,6.014125e+12,0.64
4,2001-05-16,2001.03,NaT,連,1.342442e+13,8.701310e+11,9.722730e+11,4.712950e+11,127.9,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1956.57,40.6,1.751943e+13,7.114567e+12,6.162656e+12,0.63
5,2002-05-13,2002.03,NaT,連,1.510630e+13,1.123470e+12,1.113524e+12,6.158240e+11,170.7,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2059.94,36.8,1.988894e+13,7.325072e+12,6.527956e+12,0.79
6,2003-05-08,2003.03,NaT,U,1.550155e+13,1.271646e+12,1.226652e+12,7.509420e+11,211.3,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2063.43,35.3,2.015297e+13,7.121000e+12,7.219896e+12,1.02
7,2004-05-11,2004.03,NaT,U,1.729476e+13,1.666890e+12,1.765793e+12,1.162098e+12,342.9,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2456.08,37.1,2.204023e+13,8.178567e+12,8.326215e+12,0.92
8,2005-05-10,2005.03,NaT,U,1.855153e+13,1.672187e+12,1.754637e+12,1.171260e+12,355.4,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2767.67,37.2,2.433501e+13,9.044950e+12,9.332176e+12,0.95
9,2006-05-10,2006.03,NaT,U,2.103691e+13,1.878342e+12,2.087360e+12,1.372180e+12,421.8,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3257.63,36.8,2.873160e+13,1.056045e+13,1.045979e+13,0.98


# 不要テーブル

## tables[7] 銘柄概要

In [8]:
tables[7]

,7203,トヨタ自動車,東証１,15:00


##  tables[9] 銘柄概要

In [11]:
tables[9]

,業績：
0,輸送用機器
1,単位 100株


## tables [10] ＰＥＲ ＰＢＲ 利回り 信用倍率 (データ取得日時点?)

In [12]:
tables[10]

,ＰＥＲ,ＰＢＲ,利回り,信用倍率
0,10.6倍,1.13倍,－％,1.53倍


## tables[23] 過去最高 【実績】

In [33]:
tables[23]

,Unnamed: 0,売上高,営業益,経常益,最終益,１株益
0,過去最高,28403118.00,2853971.00,2983381.00,2312694.00,741.40
1,決算期,2016.03,2016.03,2016.03,2016.03,2016.03


## tables[24] 下期業績 (過去3年 + 今年予想 + 前年同期比)

In [34]:
tables[24]

,決算期,売上高,営業益,経常益,最終益,１株益,１株配,発表日
0,U 14.10-03,14288924.0,1398618.0,1383654.0,1046502.0,331.3,125,15/05/08
1,U 15.10-03,14311630.0,1270566.0,1308230.0,1054582.0,338.1,110,16/05/11
2,U 16.10-03,14526660.0,877507.0,1017291.0,884936.0,292.6,110,17/05/10
3,U 予 17.10-03,14308793.0,903459.0,997827.0,878672.0,298.6,－,17/11/07
4,前年同期比,-1.5,3.0,-1.9,-0.7,2.1,NaN,(%)


In [50]:
tables[24].columns

Index(['決算期', '売上高', '営業益', '経常益', '最終益', '１株益', '１株配', '発表日'], dtype='object')

## tables[25] 第２四半期累計決算【実績】 (過去3年 + 前年同期比)

In [35]:
tables[25]

,決算期,売上高,営業益,経常益,最終益,１株益,対通期進捗率,発表日
0,U 15.04-09,14091488.0,1583405.0,1675151.0,1258112.0,399.4,56.1,15/11/05
1,U 16.04-09,13070533.0,1116865.0,1176534.0,946173.0,311.1,53.6,16/11/08
2,U 17.04-09,14191207.0,1096541.0,1252173.0,1071328.0,359.6,55.7,17/11/07
3,前年同期比,8.6,-1.8,6.4,13.2,15.6,NaN,(%)


In [278]:
kabupro.ix[(kabupro['証券コード'] == code) & (kabupro['会計基準'] == '米国基準') & (kabupro['決算期間'] == '第2四半期'), 
           ['連結個別', '期首', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']].tail(3)

,連結個別,期首,売上高,営業利益,経常利益,純利益,一株当り純利益,情報公開日 (更新日)
70501,連結,2015-04-01,1.409149e+13,1.583405e+12,1.675151e+12,1.258112e+12,399.39,2016-11-08
70505,連結,2016-04-01,1.307053e+13,1.116865e+12,1.176534e+12,9.461730e+11,311.08,2017-11-07
70509,連結,2017-04-01,1.419121e+13,1.096541e+12,1.252173e+12,1.071328e+12,359.55,2017-11-07


In [294]:
kabupro.ix[(kabupro['証券コード'] == code)& (kabupro['会計基準'] == '米国基準') & (kabupro['決算期間'].isin(['第2四半期', '通期'])), # 
           ['連結個別', '期首', '決算期間', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益', '情報公開日 (更新日)']].tail(5)

,連結個別,期首,決算期間,売上高,営業利益,経常利益,純利益,一株当り純利益,情報公開日 (更新日)
70501,連結,2015-04-01,第2四半期,1.409149e+13,1.583405e+12,1.675151e+12,1.258112e+12,399.39,2016-11-08
70503,連結,2015-04-01,通期,2.840312e+13,2.853971e+12,2.983381e+12,2.312694e+12,741.36,2017-05-10
70505,連結,2016-04-01,第2四半期,1.307053e+13,1.116865e+12,1.176534e+12,9.461730e+11,311.08,2017-11-07
70507,連結,2016-04-01,通期,2.759719e+13,1.994372e+12,2.193825e+12,1.831109e+12,605.47,2017-05-10
70509,連結,2017-04-01,第2四半期,1.419121e+13,1.096541e+12,1.252173e+12,1.071328e+12,359.55,2017-11-07


In [295]:
tables[12].tail(1)

,会計基準,決算期,売上高,営業益,経常益,最終益,修正日
25,U,2018.03,2.850000e+13,2.000000e+12,2.250000e+12,1.950000e+12,2017-11-07


In [42]:
# 比較参照用
kabupro.columns

Index(['証券コード', '企業名', '会計基準', '連結個別', '決算期', '決算期間', '期首', '期末',
       '名寄前勘定科目 (売上高欄)', '売上高', '営業利益', '経常利益', '純利益', '一株当り純利益',
       '希薄化後一株当り純利益', '純資産又は株主資本', '総資産', '一株当り純資産', '営業キャッシュフロー',
       '投資キャッシュフロー', '財務キャッシュフロー', '情報公開日 (更新日)'],
      dtype='object')

## おしまい

In [38]:
tables[29]

IndexError: list index out of range